# Computing the bicharacter of huge spaces of polynomials

*TODO* : Add intro

In [1]:
%display latex

%runfile diagonal_polynomial_ring.py
%runfile subspace.py
%runfile young_idempotent.py
%runfile add_degree.py
%runfile diagram.py

SymmetricFunctions(QQ).inject_shorthands(verbose=False)

#%runfile compute_character.py

## Code

Computation of the Vandermonde determinant associated to the diagram $\gamma$, where $\gamma$ is a partation (Partition) or a list of cells (Diagram). 

*Note : For now, only works for diagrams that are partitions or composition diagrams.*

In [2]:
def vandermonde(gamma):
    n = gamma.size()
    P = DiagonalPolynomialRing(QQ, n, 1, inert=1)
    X = P.variables()
    Theta = P.inert_variables()
    return matrix([[x**i[1]*theta**i[0] for i in gamma.cells()] 
                   for x,theta in zip(X[0],Theta[0])]).determinant()

def degree_vandermonde(gamma):
    return sum([i[1] for i in gamma.cells()])

Functions used to get the lists of operators by degrees for all the non inert variables of the given polynomial ring. 

In [1]:
def deriv(x, k=1):
    def f(p):
        return derivative(p, x, k)
    return f

def partial_derivatives(P):
    n = P._n
    r = P._r
    D = P._grading_set
    X = P.variables()
    op = {}
    for i in range(r):
        op[D((-1 if j==i else 0 for j in range(r)))] = [deriv(X[i,k]) for k in range(n)]
    return op

def steenrod_operators(P, degree=1):
    r = P._r
    D = P._grading_set
    op = {}
    for i in range(r):
        op[D((-degree if j==i else 0 for j in range(r)))] = [functools.partial(P.steenrod_op, i=i, k=degree+1)]
    return op

def polarization_operators(P, side=None, row_symmetry=None, max_deg=0):
    n = P._n
    r = P._r
    D = P._grading_set
    if max_deg==0:
        max_deg=n
    return {D([-d if i==i1 else 1 if i==i2 else 0 for i in range(r)]):
            [functools.partial(P.polarization, i1=i1, i2=i2, d=d, row_symmetry=row_symmetry)]
            for d in range(1, max_deg+1)
            for i1 in range(0, r)
            for i2 in range(0, r)
            if (i1<i2 if side == 'down' else i1!=i2)
           }

def symmetric_derivatives(P, list_deg, row_symmetry=None):
    D = P._grading_set
    return {D(-i for i in d) : [functools.partial(P.symmetric_derivative, d=d, row_symmetry=row_symmetry)] 
            for d in list_deg}

def merge(dict1, dict2):
    result = dict1
    for key, value in dict2.iteritems():
        if key in result:
            result[key] += value
        else:
            result[key] = value
    return result

Projection on the isotypic components; take as parameter a subspace and an integer or a partition. 

In [3]:
def Isotyp(S, arg):
    if isinstance(arg, Partition):
        list_partitions = [arg]
    elif isinstance(arg, Integer):
        list_partitions = Partitions(arg)
    else : 
        print("Error: arg should be a partition or an integer.")
    
    basis = S.basis()
    result = {}
    for nu in list_partitions:
        for key, value in basis.iteritems():
            gen = [apply_young_idempotent(p, nu) for p in value]
            basis_nu = Subspace(gen, {}).basis()
            if basis_nu != {} :
                result[(key, tuple(nu))] = basis_nu[0]
    return Subspace(result, operators={})

def add_degrees_isotypic(gen_deg, op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    return D(gen_deg[0])+D(op_deg), gen_deg[1]

Create the polarized space given by the input operators.

In [5]:
def PolarizedSpace(pol_ring, S, operators):
    basis = S.basis()
    P = pol_ring
    r = P._r
    D = P._grading_set
    
    generators = {}
    for key, value in basis.iteritems():
        d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
        generators[d] = [P(b) for b in value]
    return Subspace(generators, operators, add_degrees=add_degrees_isotypic)

Compute the image of S by the given operators.

In [4]:
def Range(S, operators, add_degrees=add_degrees_isotypic):
    result = {}
    basis = S.basis()
    for key, b in basis.iteritems():
        result = merge(result, {add_degrees(key, deg): 
                                     [op(p) for p in b for op in op_list if op(p)!=0] 
                                     for deg, op_list in operators.iteritems()})    
    if result != {} :
        return Subspace(result, {}, add_degrees)
    else :
        return None

Computation of the character from a subspace already projected on the isotypic components. 

In [7]:
def character(S, n, r, left_basis=s, right_basis=s, row_symmetry=None):
    basis = S.basis()
    charac = 0
    if row_symmetry != "permutation":
        q = PolynomialRing(QQ,'q',r).gens()
        
    for nu in Partitions(n):
        basis_nu = {}
        charac_nu = 0
        # Get the nu_isotypic part of the basis
        for key, value in basis.iteritems():
            if Partition(key[1]) == nu:
                basis_nu[key[0]] = value
        
        # Use monomials to compute the character
        if row_symmetry == "permutation":
            for deg, b in basis_nu.iteritems():
                charac_nu += sum(m(Partition(deg)) for p in b)
            if charac_nu != 0 :
                if left_basis == s :
                    charac_nu = s(charac_nu).restrict_partition_lengths(r,exact=False)
                elif left_basis != m :
                    charac_nu = left_basis(charac_nu)
                
        # Or use directly the degrees
        else:
            for deg, b in basis_nu.iteritems():
                charac_nu += sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
            if charac_nu != 0 :
                if left_basis == s :
                    charac_nu = s.from_polynomial(charac_nu).restrict_partition_lengths(r,exact=False)           
                else:
                    charac_nu = left_basis.from_polynomial(charac_nu)
                
        # Make the tensor product with s[nu]
        if charac_nu != 0:
            charac += tensor([charac_nu, right_basis(s(nu))])
    return charac

Factorisation of the tensorial products by represention of $S_n$. And computation of the dimensions. 

In [9]:
def factorise(f, n):
    SymmetricFunctions(QQ).s()
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu])) for mu in result.keys()]
    for a, b in result2:
        if b!=0:
            print a
            show(b)
        
def dimension(f, n):
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu]).expand(1, alphabet=['q'])) for mu in result.keys()]
    q = result2[0][1].parent().gens()[0]
    return [(tuple(a), b.subs({q:1})) for a,b in result2]

## Examples

### Partition 21

Computation of the Vandermonde determinant $\mathcal{V}_\lambda$ associated to the partition $\lambda = 21$.

In [9]:
v = vandermonde(Partition([2,1]))
v

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

Space generated by $\mathcal{V}_\lambda$ and its partials derivatives.

In [10]:
P1 = DiagonalPolynomialRing(QQ, 3, 1, inert=1)
generator = {P1._grading_set((1,)) : [P1(v)]}
list_op = partial_derivatives(P1)
V1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V1.basis()

{(0,): (theta01 - theta02, theta00 - theta02),
 (1,): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

Projection on the isotypic components.

In [11]:
V1_iso = Isotyp(V1, 3)
V1_iso.basis()

{((0,), (2, 1)): (-theta00 + theta02,),
 ((1,),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

We add another set of variables and we polarize.

In [12]:
P2 = DiagonalPolynomialRing(QQ, 3, 2, inert=1)
op_pol = polarization_operators(P2)
V1_pol = PolarizedSpace(P2, V1_iso, op_pol)
V1_pol.basis()

{((0, 0), (2, 1)): (theta00 - theta02,),
 ((0, 1),
  (1,
   1,
   1)): (-x11*theta00 + x12*theta00 + x10*theta01 - x12*theta01 - x10*theta02 + x11*theta02,),
 ((1, 0),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

Computation of the bicharacter.

In [13]:
character(V1_pol, 3, 2)

s[] # s[2, 1] + s[1] # s[1, 1, 1]

### Diagram (0,1,3)

In [13]:
v = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
v.factor()

(x01 - x02) * (-x00 + x01) * (x00 - x02) * (x00 + x01 + x02)

We create the subspace generated by $\mathcal{V}_\gamma$ and its partial derivatives. We can add the Steenrod operators without any change in the result.

In [15]:
P1 = DiagonalPolynomialRing(QQ, 3, 1)
generator = {P1._grading_set((v.degree(),)) : [P1(v)]}
list_op = merge(merge(partial_derivatives(P1), steenrod_operators(P1, 1)), steenrod_operators(P1, 2))

In [16]:
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (-x00*x01 + x00*x02, -x00*x01 + x01*x02, x00^2 - x02^2, x01^2 - x02^2),
 (3,): (1/3*x00^3 - 1/3*x01^3 - x00*x02^2 + x01*x02^2,
  -x00^2*x01 + 1/3*x01^3 + x00^2*x02 - 1/3*x02^3,
  -1/3*x00^3 + x00*x01^2 - x01^2*x02 + 1/3*x02^3),
 (4,): (x00^3*x01 - x00*x01^3 - x00^3*x02 + x01^3*x02 + x00*x02^3 - x01*x02^3,)}

In [17]:
character(Isotyp(W1, 3), 3, 1)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Apply $\sum_i \partial_{x_i}$ and $\sum_i x_i \partial_{x_i}^2$ à $\mathcal{V}_{\gamma}$. 

In [18]:
x00, x01, x02 = P1.variables()[0]
P1.symmetric_derivative(P1(v), (1,))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [19]:
P1.steenrod_op(P1(v), 0, 2)

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

Polarization

In [20]:
P2 = DiagonalPolynomialRing(QQ, 3, 2)
op_pol = polarization_operators(P2)
W2 = PolarizedSpace(P2, Isotyp(W1, 3), op_pol)
character(W2, 3, 2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

In [21]:
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(P2._r)) 
                for k1 in range(1, 3+1) for i1 in range(0, P2._r)]
#list_degrees += [(1,1),(1,2),(1,3),(2,1),(2,2),(2,3),(3,1),(3,2),(3,3)]
sym_diff = symmetric_derivatives(P2, list_degrees)
#sym_diff = merge(merge(symmetric_derivatives(P2, list_degrees), 
#                       steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
character(Range(W2, sym_diff), 3, 2)

s[] # s[3] + s[1] # s[2, 1] + s[1, 1] # s[1, 1, 1] + s[2] # s[2, 1] + s[3] # s[1, 1, 1]

In [22]:
charac = character(W2, 3, 2) - character(Range(W2, sym_diff), 3, 2)
charac

s[1] # s[3] + s[1, 1] # s[2, 1] + s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Or in terms of monomial symmetric functions.

In [23]:
character(W2, 3, 2, left_basis=m) - character(Range(W2, sym_diff), 3, 2, left_basis=m)

m[1] # s[3] + 2*m[1, 1] # s[2, 1] + m[2] # s[2, 1] + m[2, 1] # s[1, 1, 1] + m[2, 1] # s[2, 1] + m[2, 2] # s[1, 1, 1] + m[3] # s[2, 1] + m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]

We factorise the character in terms of representations of $S_3$.

In [24]:
factorise(charac, 3)

[1, 1, 1]


s[2, 1] + s[4]

[3]


s[1]

[2, 1]


s[1, 1] + s[2] + s[3]

### Partition 311

In [42]:
P1 = DiagonalPolynomialRing(QQ, 5, 1, inert=1)
v = vandermonde(Partition([3,1,1]))
deg_v = 3
generator = {P1._grading_set((deg_v,)) : [P1(v)]}
list_op = partial_derivatives(P1)
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

P2 = DiagonalPolynomialRing(QQ, 5, 2, inert=1)
op_pol = polarization_operators(P2)
W2 = PolarizedSpace(P2, Isotyp(W1, 5), op_pol)
character(W2, 5, 2)

s[] # s[3, 1, 1] + s[1] # s[2, 1, 1, 1] + s[1] # s[2, 2, 1] + s[1, 1] # s[1, 1, 1, 1, 1] + s[2] # s[2, 1, 1, 1] + s[3] # s[1, 1, 1, 1, 1]

### Others cases

#### Diagramme $\{(0,0),(0,1),(1,1),(0,2)\}$ et autres

Pas de quotient dans ce cas.

In [74]:
n = 4
P1 = DiagonalPolynomialRing(QQ, n, 1, inert=1)
gamma = Partition([2,2])
v = vandermonde(gamma)
show(v)
deg_v = degree_vandermonde(gamma)
generator = {P1._grading_set((deg_v,)) : [P1(v)]}
list_op = partial_derivatives(P1)
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
show(character(Isotyp(W1, n), n, 1))

r = 3
P2 = DiagonalPolynomialRing(QQ, n, r, inert=1)
op_pol = polarization_operators(P2)
W1 = PolarizedSpace(P2, Isotyp(W1, n), op_pol)
character(W1, n, r)

x00*x02*theta00*theta01 - x01*x02*theta00*theta01 - x00*x03*theta00*theta01 + x01*x03*theta00*theta01 - x00*x01*theta00*theta02 + x01*x02*theta00*theta02 + x00*x03*theta00*theta02 - x02*x03*theta00*theta02 + x00*x01*theta01*theta02 - x00*x02*theta01*theta02 - x01*x03*theta01*theta02 + x02*x03*theta01*theta02 + x00*x01*theta00*theta03 - x00*x02*theta00*theta03 - x01*x03*theta00*theta03 + x02*x03*theta00*theta03 - x00*x01*theta01*theta03 + x01*x02*theta01*theta03 + x00*x03*theta01*theta03 - x02*x03*theta01*theta03 + x00*x02*theta02*theta03 - x01*x02*theta02*theta03 - x00*x03*theta02*theta03 + x01*x03*theta02*theta03

s[] # s[2, 2] + s[1] # s[2, 1, 1] + s[2] # s[1, 1, 1, 1]

s[] # s[2, 2] + s[1] # s[2, 1, 1] + s[2] # s[1, 1, 1, 1]

In [73]:
n = 5
P1 = DiagonalPolynomialRing(QQ, n, 1, inert=1)
gamma = Partition([2,2,1])
v = vandermonde(gamma)
deg_v = degree_vandermonde(gamma)
generator = {P1._grading_set((deg_v,)) : [P1(v)]}
list_op = partial_derivatives(P1)
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
show(character(Isotyp(W1, n), n, 1))

r = 4
P2 = DiagonalPolynomialRing(QQ, n, r, inert=1)
op_pol = polarization_operators(P2)
W1 = PolarizedSpace(P2, Isotyp(W1, n), op_pol)
character(W1, n, r)

s[] # s[2, 2, 1] + s[1] # s[2, 1, 1, 1] + s[2] # s[1, 1, 1, 1, 1]

s[] # s[2, 2, 1] + s[1] # s[2, 1, 1, 1] + s[2] # s[1, 1, 1, 1, 1]

In [54]:
gamma = Partition([2,2])
n = gamma.size()
P1 = DiagonalPolynomialRing(QQ, n, 1, inert=1)
v = vandermonde(gamma)
deg_v = degree_vandermonde(gamma)
generator = {P1._grading_set((deg_v,)) : [P1(v)]}
list_op = partial_derivatives(P1)
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
character(Isotyp(W1, n), n, 1)

P2 = DiagonalPolynomialRing(QQ, n, 3, inert=1)
op_pol = polarization_operators(P2)
W1 = PolarizedSpace(P2, Isotyp(W1, n), op_pol)
factorise(character(W1, n, 3), 4)

[1, 1, 1, 1]


s[2]

[2, 2]


s[]

[2, 1, 1]


s[1]